# Neural Networks

In [12]:
import pandas as pd
import numpy as np

from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfVectorizer
)
from sklearn.metrics import f1_score
from ast import literal_eval


# Path to train and test files
train_path = '../data/train_E6oV3lV.csv'
test_path = '../data/test_tweets_anuFYb8.csv'

train  = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# Processed data
df = pd.read_csv('../data/pandas_data_frame.csv', index_col=0)
all_data = df.where((pd.notnull(df)), '')
all_data['hashtag'] = all_data['hashtag'].apply(literal_eval)


# bag-of-words feature matrix
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow = bow_vectorizer.fit_transform(all_data['tidy_tweet'])

train_bow = bow[:31962,:]
test_bow = bow[31962:,:]

# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, train['label'], random_state=42, test_size=0.3)


# TF-IDF feature matrix
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(all_data['tidy_tweet'])

train_tfidf = tfidf[:31962,:]
test_tfidf = tfidf[31962:,:]

xtrain_tfidf = train_tfidf[ytrain.index]
xvalid_tfidf = train_tfidf[yvalid.index]

# Stack BOW and TF-IDF
x_val = hstack([xvalid_tfidf,xvalid_bow])
x_train = hstack([xtrain_tfidf,xtrain_bow])

In [8]:
# Merge BOW and TF-IDF and aply PCA

x_train = x_train.toarray()
x_test = x_val.toarray()
y_train = ytrain
y_test = yvalid

pca = decomposition.PCA(n_components=0.95, svd_solver='full')
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(22373, 833)
(22373,)
(9589, 833)
(9589,)


In [18]:
conteo_mayusculas = count_caps(all_data['tidy_tweet'])

In [19]:
np.count_nonzero(conteo_mayusculas)

0

In [ ]:
from keras import backend as K
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import Adam, RMSprop
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint

import sys
sys.path.append("../") 
from personal_library.sce_keras.callbacks import (
    F1PrecisionRecall,
    LearningRateDecay
)
from personal_library.sce_keras.loss_functions import f1_loss
from personal_library.sce_keras.metrics_functions import f1


num_classes = 1
batch_size = 32
epochs = 1
learnRate = 0.001

metric = F1PrecisionRecall()
lrate_decay = LearningRateDecay(epochs, learnRate).poly_decay

#Define model architecture
model = Sequential()
model.add( Dense( 2048, activation='relu', input_shape=(x_train.shape[1],) ) )
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(1024))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(512))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(256))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(num_classes, activation='sigmoid'))
#model.summary()

checkpoint_path = "../model_wehigts/Wehigts.hdf5"
checkpointer = ModelCheckpoint(filepath=checkpoint_path,
                               monitor='val_loss', verbose=1,
                               save_best_only=True, mode='min')

adam = Adam(lr=learnRate, beta_1=0.9, beta_2=0.999,
            epsilon=None, decay=1e-6, amsgrad=False)
rms = RMSprop(lr=learnRate, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss=f1_loss, 
            optimizer=adam, 
            metrics=['accuracy', f1]) 

history = model.fit(x_train, y_train, 
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test),
            callbacks=[checkpointer, metric, lrate_decay])


In [13]:
#Load best model
model.load_weights(checkpoint_path)
y_pred = model.predict(x_test, batch_size=1)
y_pred = np.where(y_pred > 0.5, 1, 0)

print("Neural Network f1_sklearn: {}".format(f1_score(y_test, y_pred)))

Neural Network f1_sklearn: 0.34768740031897927
